In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv, pd.read_parquet )
import polars as pl

import os, gc
from tqdm.auto import tqdm
import pickle # module to serialize and deserialize objects
import re # for Regular expression operations 

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data  import Dataset, DataLoader
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer)
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, Timer

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor

from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
#from sklearn import svm 
import xgboost as xgb
from sklearn.linear_model import BayesianRidge,Ridge
import random

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import kaggle_evaluation.jane_street_inference_server

In [2]:
%%time
path = "/kaggle/input/jane-street-real-time-market-data-forecasting"
# samples = [] 

# Load a data from each file:
# r = range(8,10)
# for i in r:
#     file_path = f"{path}/train.parquet/partition_id={i}/part-0.parquet"
#     part = pd.read_parquet(file_path)
#     samples.append(part)
    
#sample_df = pd.concat(samples, ignore_index=True) # Concatenate all samples into one DataFrame if needed

#sample_df.round(1)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


We can see that two files (0 and 1) has a total of 4 748 457 rows. 
I have used pandas to load it and it took almost 9 sec. Th ere are a total of 339 days  (about one years of trading data).  

In [3]:
# numerical_features=[]
# numerical_features=sample_df.filter(regex='^responder_').columns.tolist() # Separate responders
# numerical_features.remove('responder_6')

In [4]:
ENSEMBLE_SOLUTIONS = ['SOLUTION_14','SOLUTION_5']
OPTION,__WTS = 'option 91',[0.89, 0.28]
# 0,988

In [5]:
def predict(test:pl.DataFrame, lags:pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:    
    pdB = predict_14(test,lags).to_pandas() # pred are lower then in C
    pdC = predict_5(test,lags).to_pandas() 

    pdB = pdB.rename(columns={'responder_6':'responder_B'})
    pdC = pdC.rename(columns={'responder_6':'responder_C'})
    pds = pd.merge(pdB,pdC, on=['row_id'])
    pds['responder_6'] =\
        pds['responder_B'] *__WTS[0] +\
        pds['responder_C'] *__WTS[1] 

    display(pds)
    predictions = test.select('row_id', pl.lit(0.0).alias('responder_6'))
    pred = pds['responder_6'].to_numpy()
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))
    return predictions

 5. [JS Ridge baseline](https://www.kaggle.com/code/yunsuxiaozi/js-ridge-baseline) Lb=0.0026
 [yunsuxiaozi](https://www.kaggle.com/yunsuxiaozi)

In [6]:
# if 'SOLUTION_7' in ENSEMBLE_SOLUTIONS:

    
#     def seed_everything(seed):
#         np.random.seed(seed)
#         random.seed(seed)
        
#     seed_everything(seed=2024)
    
#     def custom_metric(y_true,y_pred,weight):
#         weighted_r2=1-(np.sum(weight*(y_true-y_pred)**2)/np.sum(weight*y_true**2))
#         return weighted_r2

#     print("read data")

#     train=pl.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9/part-0.parquet")
#     train=train.to_pandas()

#     print("get X,y")

#     cols=[f'feature_0{i}' if i<10 else f'feature_{i}' for i in range(79)]
#     X=train[cols].fillna(3).values
#     y=train['responder_6'].values
#     print("train test split")
#     split=1300000#大约是8:2
#     weights=train['weight'].values
    # train_X,train_y,test_X,test_y,train_weight,test_weight=X[:-split],y[:-split],X[-split:],y[-split:],weights[:-split],weights[-split:]
    # print(f"train_X.shape:{train_X.shape},test_X.shape:{test_X.shape}")
    # print("fit and predict")
    # model_7 = BayesianRidge()
    # model_7.fit(train_X,train_y)
    # train_pred = model_7.predict(train_X)
    # test_pred  = model_7.predict(test_X)
    # print(f"train weighted_r2:{custom_metric(train_y, train_pred, weight=train_weight)}")
    # print(f"test weighted_r2: {custom_metric(test_y,  test_pred,  weight= test_weight)}")
    


In [7]:
# import joblib
# joblib.dump(model_7, 'ridge_b.pkl')

In [8]:
# if 'SOLUTION_7' in ENSEMBLE_SOLUTIONS:
#     def predict_7(test,lags):
#         cols=[f'feature_0{i}' if i<10 else f'feature_{i}' for i in range(79)]
#         predictions = test.select(
#             'row_id',
#             pl.lit(0.0).alias('responder_6'),
#         )
#         test_preds=model_7.predict(test[cols].to_pandas().fillna(3).values)
#         predictions = predictions.with_columns(pl.Series('responder_6', test_preds.ravel()))
#         return predictions


# if 'SOLUTION_7' in ENSEMBLE_SOLUTIONS:
#     from sklearn.linear_model import BayesianRidge
#     import joblib
#     model_7 = joblib.load('/kaggle/input/bayes_ridge/scikitlearn/default/1/ridge_b.pkl')

In [9]:
if 'SOLUTION_5' in ENSEMBLE_SOLUTIONS:
    
    def predict_5(test,lags):
        cols=[f'feature_0{i}' if i<10 else f'feature_{i}' for i in range(79)]
        predictions = test.select(
            'row_id',
            pl.lit(0.0).alias('responder_6'),
        )
        test_preds=model_5.predict(test[cols].to_pandas().fillna(3).values)
        predictions = predictions.with_columns(pl.Series('responder_6', test_preds.ravel()))
        return predictions

if 'SOLUTION_5' in ENSEMBLE_SOLUTIONS:
    from sklearn.linear_model import BayesianRidge
    import joblib
    model_5 = joblib.load('/kaggle/input/jane-street-5-and-7_/other/default/1/ridge_model_5(1).pkl')

14. [Jane Street RMF NN + XGB](https://www.kaggle.com/code/voix97/jane-street-rmf-nn-xgb), Lb=0.0056
 [Xiang Sheng](https://www.kaggle.com/voix97)

### NN + XGB inference

### Configurations

In [10]:
if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:    
    
    class CONFIG:
        seed = 42
        target_col = "responder_6"
        # feature_cols = ["symbol_id", "time_id"] + [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
        feature_cols = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
        feature_ridge = [f'feature_0{i}' if i<10 else f'feature_{i}' for i in range(79)]
        model_paths = [
            #"/kaggle/input/js24-train-gbdt-model-with-lags-singlemodel/result.pkl",
            #"/kaggle/input/js24-trained-gbdt-model/result.pkl",
            "/kaggle/input/js-xs-nn-trained-model",
            "/kaggle/input/js-with-lags-trained-xgb/result.pkl",
        ]

if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
    
    valid = pl.scan_parquet(
        f"/kaggle/input/js24-preprocessing-create-lags/validation.parquet/"
    ).collect().to_pandas()

### Load model

In [11]:
if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS: 
    
    xgb_model = None
    model_path = CONFIG.model_paths[1]
    with open( model_path, "rb") as fp:
        result = pickle.load(fp)
        xgb_model = result["model"]

    xgb_feature_cols = ["symbol_id", "time_id"] + CONFIG.feature_cols

    # Show model
    #display(xgb_model)

if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
    
    # Custom R2 metric for validation
    def r2_val(y_true, y_pred, sample_weight):
        r2 = 1 - np.average((y_pred - y_true) ** 2, weights=sample_weight) / (np.average((y_true) ** 2, weights=sample_weight) + 1e-38)
        return r2


    class NN(LightningModule):
        def __init__(self, input_dim, hidden_dims, dropouts, lr, weight_decay):
            super().__init__()
            self.save_hyperparameters()
            layers = []
            in_dim = input_dim
            for i, hidden_dim in enumerate(hidden_dims):
                layers.append(nn.BatchNorm1d(in_dim))
                if i > 0:
                    layers.append(nn.SiLU())
                if i < len(dropouts):
                    layers.append(nn.Dropout(dropouts[i]))
                layers.append(nn.Linear(in_dim, hidden_dim))
                # layers.append(nn.ReLU())
                in_dim = hidden_dim
            layers.append(nn.Linear(in_dim, 1))  
            layers.append(nn.Tanh())
            self.model = nn.Sequential(*layers)
            self.lr = lr
            self.weight_decay = weight_decay
            self.validation_step_outputs = []

        def forward(self, x):
            return 5 * self.model(x).squeeze(-1)  

        def training_step(self, batch):
            x, y, w = batch
            y_hat = self(x)
            loss = F.mse_loss(y_hat, y, reduction='none') * w  
            loss = loss.mean()
            self.log('train_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
            return loss

        def validation_step(self, batch):
            x, y, w = batch
            y_hat = self(x)
            loss = F.mse_loss(y_hat, y, reduction='none') * w
            loss = loss.mean()
            self.log('val_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
            self.validation_step_outputs.append((y_hat, y, w))
            return loss

        def on_validation_epoch_end(self):
            """Calculate validation WRMSE at the end of the epoch."""
            y = torch.cat([x[1] for x in self.validation_step_outputs]).cpu().numpy()
            if self.trainer.sanity_checking:
                prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
            else:
                prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
                weights = torch.cat([x[2] for x in self.validation_step_outputs]).cpu().numpy()
                # r2_val
                val_r_square = r2_val(y, prob, weights)
                self.log("val_r_square", val_r_square, prog_bar=True, on_step=False, on_epoch=True)
            self.validation_step_outputs.clear()

        def configure_optimizers(self):
            optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5,
                                                                   verbose=True)
            return {
                'optimizer': optimizer,
                'lr_scheduler': {
                    'scheduler': scheduler,
                    'monitor': 'val_loss',
                }
            }

        def on_train_epoch_end(self):
            if self.trainer.sanity_checking:
                return
            epoch = self.trainer.current_epoch
            metrics = {k: v.item() if isinstance(v, torch.Tensor) else v for k, v in self.trainer.logged_metrics.items()}
            formatted_metrics = {k: f"{v:.5f}" for k, v in metrics.items()}
            print(f"Epoch {epoch}: {formatted_metrics}")

In [12]:
if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
    
    N_folds = 5
    models = []
    for fold in range(N_folds):
        checkpoint_path = f"{CONFIG.model_paths[0]}/nn_{fold}.model"
        model = NN.load_from_checkpoint(checkpoint_path) #, strict=False
        models.append(model.to("cuda:0"))

### CV Score

In [13]:
# if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
    
#     valid = pl.scan_parquet(
#         f"/kaggle/input/js24-preprocessing-create-lags/validation.parquet/"
#     ).collect().to_pandas()

In [14]:
# if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS: 
#     X_valid = valid[ xgb_feature_cols ]
#     y_valid = valid[ CONFIG.target_col ]
#     w_valid = valid[ "weight" ]
#     y_pred_valid_xgb = xgb_model.predict(X_valid)
#     #y_pred_valid_xgb = y_pred_valid_xgb + (y_pred_valid_xgb * w_valid).sum()/ w_valid.sum()
#     #y_pred_valid_xgb = y_pred_valid_xgb + y_pred_valid_xgb.mean()
#     valid_score = r2_score( y_valid, y_pred_valid_xgb, sample_weight=w_valid )
#     valid_score

In [15]:
# if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
#     X_valid = valid[ CONFIG.feature_cols ]
#     y_valid = valid[ CONFIG.target_col ]
#     w_valid = valid[ "weight" ]
#     X_valid = X_valid.fillna(method = 'ffill').fillna(0)
#     X_valid.shape, y_valid.shape, w_valid.shape

# if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
#     y_pred_valid_nn = np.zeros(y_valid.shape)
#     with torch.no_grad():
#         for model in models:
#             model.eval()
#             y_pred_valid_nn += model(torch.FloatTensor(X_valid.values).to("cuda:0")).cpu().numpy() / len(models)
#     valid_score = r2_score( y_valid, y_pred_valid_nn, sample_weight=w_valid )
#     valid_score

# if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
#     #y_pred_valid_ensemble = 0.5 * (y_pred_valid_xgb + y_pred_valid_nn)
#     # print('XGB: ', y_pred_valid_xgb.mean(), 'NN: ', y_pred_valid_nn.mean())
#     # print('XGB median: ', np.median(y_pred_valid_xgb), 'NN median: ', np.median(y_pred_valid_nn))
#     y_pred_valid_ensemble = 0.65 * y_pred_valid_xgb + 0.35*y_pred_valid_nn 
#     #print('Ensemble: ', y_pred_valid_ensemble.mean())
#     #print('Ensemble median: ', np.median(y_pred_valid_ensemble))
#     #y_pred_valid_ensemble = y_pred_valid_ensemble -(y_pred_valid_ensemble * w_valid).sum()/ w_valid.sum()
#     #(y_pred_valid_ensemble).mean()
#     valid_score = r2_score( y_valid, y_pred_valid_ensemble, sample_weight=w_valid )
#     valid_score

# if 'SOLUTION_5' in ENSEMBLE_SOLUTIONS:
#     X_valid = valid[CONFIG.feature_ridge]
#     y_valid = valid[CONFIG.target_col]
#     w_valid = valid[ "weight" ] 
#     X_valid = X_valid.fillna(method = 'ffill').fillna(0)
#     X_valid.shape, y_valid.shape, w_valid.shape
#     y_pred_valid_ridge = model_5.predict(X_valid.fillna(method = 'ffill').values)
#     y_pred_valid_ensemble = y_pred_valid_ensemble * 0.98 + 0.28*y_pred_valid_ridge
#     y_pred_valid_ensemble = y_pred_valid_ensemble - (y_pred_valid_ensemble * w_valid).sum()/w_valid.sum()
#     valid_score = r2_score( y_valid, y_pred_valid_ensemble, sample_weight=w_valid )
#     valid_score

# if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:
#     del valid, X_valid, y_valid, w_valid
#     gc.collect()

In [16]:
# valid_score

In [17]:
# Scores in validation for different post_processing

# 0.01131019 score for ensemble_pred - weighted avg(ensemble_pred) (0.35xgb+0.55nn)
# 0.01188395 score for ensemble_pred - weighted avg(ensemble_pred) (0.4xgb+0.6nn)
# 0.01199055 score for ensemble_pred - weighted avg(ensemble_pred) (0.5 weights for xgb+nn)
# 0.01201276 score for pred_xgb + avg(pred_xgb)
# 0.01201298 score for pred_xgb + weighted avg(pred_xgb)
# 0.01202971 score for pred_xgb + meadian(pred_xgb)
# 0.01202981 score for usual predictions
# 0.01204447 score for pred_xgb - weighted avg(pred_xgb)
# 0.01204464 score for pred_xgb - avg(pred_xgb)
# 0.01206211 score for pred_ensamble - avg(pred_ensamble)
# 0.01211081 score 0.6 xgb +0.4 nn - avg(pred)
# 0.01211419 score 0.65 xgb +0.35 nn - avg(pred)
# Лучше чуть увеличить вес бустинга в ансамбле
# Вычитание среднего предсказания ансамбля, а среднее отриц. => итог. предсказание д.б. больше  

# 0.01185643 сейчас для 0.5 весов и без пост-обработки
# 0.01206463  ensemble 0.95 + 0.28ridge
# 0.01217069 ensemble 0.98 + 0.28ridge  
# 0.01230881 ensemble 0.98 + 0.28ridge - weighted avg(pred)


In [18]:
if 'SOLUTION_14' in ENSEMBLE_SOLUTIONS:    
    
    lags_ : pl.DataFrame | None = None

    def predict_14(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
        global lags_
        if lags is not None:
            lags_ = lags

        predictions_14 = test.select(
            'row_id',
            pl.lit(0.0).alias('responder_6'),
        )
        symbol_ids = test.select('symbol_id').to_numpy()[:, 0]

        if not lags is None:
            lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last() # pick up last record of previous date
            test = test.join(lags, on=["date_id", "symbol_id"],  how="left")
        else:
            test = test.with_columns(
                ( pl.lit(0.0).alias(f'responder_{idx}_lag_1') for idx in range(9) )
            )

        preds = np.zeros((test.shape[0],))
        preds += 0.65*xgb_model.predict(test[xgb_feature_cols].to_pandas()) / 2
        test_input = test[CONFIG.feature_cols].to_pandas()
        test_input = test_input.fillna(method = 'ffill').fillna(0)
        test_input = torch.FloatTensor(test_input.values).to("cuda:0")
        with torch.no_grad():
            for i, nn_model in enumerate(tqdm(models)):
                nn_model.eval()
                preds += 0.35*nn_model(test_input).cpu().numpy() / 10
        print(f"predict> preds.shape =", preds.shape)

        predictions_14 = \
        test.select('row_id').\
        with_columns(
            pl.Series(
                name   = 'responder_6', 
                values = np.clip(preds-preds.mean(), a_min = -5, a_max = 5),
                dtype  = pl.Float64,
            )
        )

        # The predict function must return a DataFrame
        #assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
        # with columns 'row_id', 'responer_6'
        #assert list(predictions.columns) == ['row_id', 'responder_6']
        # and as many rows as the test data.
        #assert len(predictions) == len(test)

        return predictions_14

In [19]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

  0%|          | 0/5 [00:00<?, ?it/s]

predict> preds.shape = (39,)


,row_id,responder_B,responder_C,responder_6
0,0,-0.006624,0.073934,0.014806
1,1,-0.015249,0.073934,0.007130
2,2,-0.005731,0.073934,0.015601
3,3,0.000901,0.073934,0.021504
4,4,0.000168,0.073934,0.020851
5,5,-0.000501,0.073934,0.020256
6,6,0.001338,0.073934,0.021892
7,7,0.002502,0.073934,0.022928
8,8,0.005828,0.073934,0.025888
9,9,-0.005325,0.073934,0.015962
